In [1]:
from data_processing import *
from correction_algorithms import *
from figures_plotting import *

/apps/brussel/RL8/skylake/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Load the historical and future data. For the future data we focus only on ssp2_rcp26_gfdl as an example.

In [2]:
hist_gridded_data = process_files('/user/brussel/105/vsc10535/SocioHydroLayers/input/sector_demand_link/processed/yearly_format/irrigation/ensemble_mean/', 'ensemble_mean_withd_irrig_')


A xarray.Dataset corresponding to the data in the given path was succesfully generated.


In [3]:
future_gridded_data = process_files('/user/brussel/105/vsc10535/SocioHydroLayers/input/sector_demand_future_link/ssp2_rcp26_gfdl/other_sectors/nc_format/yearly_format/irrigation/', 'twdirr_m3_per_day_')

A xarray.Dataset corresponding to the data in the given path was succesfully generated.


## Load the countries and US states masks.

In [4]:
mask_countries = read_mask_file('/user/brussel/105/vsc10535/SocioHydroLayers/input/support_data/updated_global_mask_v2.nc')
mask_US_states = read_mask_file('/user/brussel/105/vsc10535/SocioHydroLayers/input/support_data/updated_US_mask_v2.nc')

A numpy.ndarray corresponding to the mask file provided was succesfully generated.
A numpy.ndarray corresponding to the mask file provided was succesfully generated.


## Initialize supporting data

In [5]:
countries_names, countries_indices, states_names, states_indices = define_countries_and_states_names_and_indices()

Countries and US states names were initialized. Country index 177 corresponds to gridcells not attributed to any country. State code 51 corresponds to gridcells not attributed to any US state.


## Initialize xarray.DataArray's to contain the countries or US states historical and future time-series.

In [6]:
hist_countries_timeseries = xr.DataArray(np.zeros((len(countries_indices), 480)), dims=('country', 'time'),
                                                 coords={'country': countries_indices, 'time': pd.date_range('1971-01', periods=480, freq='M')})

future_countries_timeseries = xr.DataArray(np.zeros((len(countries_indices), 1092)), dims=('country', 'time'),
                                             coords={'country': countries_indices, 'time': pd.date_range('2010-01', periods=1092, freq='M')})

hist_US_states_timeseries = xr.DataArray(np.zeros((len(states_indices), 480)), dims=('US_states', 'time'),
                                                 coords={'US_states': states_indices, 'time': pd.date_range('1971-01', periods=480, freq='M')})

future_US_states_timeseries = xr.DataArray(np.zeros((len(states_indices), 1092)), dims=('US_states', 'time'),
                                             coords={'US_states': states_indices, 'time': pd.date_range('2010-01', periods=1092, freq='M')})

## Compute national and US states time-series.

In [7]:
compute_national_or_state_monthly_timeseries(hist_gridded_data, hist_countries_timeseries, mask_countries)

Provided data covers the years 1971--2010.
Begin computing the historical countries time-series.
Calculation completed at 25%
Calculation completed at 50%
Calculation completed at 75%
Calculation completed at 100%


In [8]:
compute_national_or_state_monthly_timeseries(hist_gridded_data, hist_US_states_timeseries, mask_US_states)

Provided data covers the years 1971--2010.
Begin computing the historical US states time-series.
Calculation completed at 25%
Calculation completed at 50%
Calculation completed at 75%
Calculation completed at 100%


In [9]:
compute_national_or_state_monthly_timeseries(future_gridded_data, future_countries_timeseries, mask_countries)

Provided data covers the years 2010--2100.
Begin computing the future countries time-series.
Calculation completed at 24%
Calculation completed at 48%
Calculation completed at 72%
Calculation completed at 96%
Calculation completed at 100%


In [10]:
compute_national_or_state_monthly_timeseries(future_gridded_data, future_US_states_timeseries, mask_US_states)

Provided data covers the years 2010--2100.
Begin computing the future US states time-series.
Calculation completed at 24%
Calculation completed at 48%
Calculation completed at 72%
Calculation completed at 96%
Calculation completed at 100%


## Plot countries and US states timeseries:

In [11]:
plot_countries_monthly_timeseries_no_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_no_correction/country_level/", hist_countries_timeseries, future_countries_timeseries, "Irrigation", countries_names)

All plots have been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_no_correction/country_level/ directory.


In [12]:
plot_US_states_monthly_timeseries_no_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_no_correction/US_state_level", hist_US_states_timeseries, future_US_states_timeseries, "Irrigation", states_names)

All plots have been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_no_correction/US_state_level directory.


## Visually inspect the national timeseries plots:
Main issues found are drops/jumps at the transition between past and future periods (year 2010-2011), and missmatch of the seasonality pattern/amplitude.

A suitable correction method is applied, which will shift the future timeseries to match historical values for year 2010, while adjusting future seasonality pattern/amplitude for continuity with historical timeseries, while preserving trends in the seasonality change projected in the future dataset.

## Initialize the xarray.DataArray's to contain the countries or US states corrected future time-series.

In [13]:
dims = ('country', 'time')
coords = {
    'country': future_countries_timeseries.country.values,
    'time': future_countries_timeseries.time.values
}

dims_states = ('US_states', 'time')
coords_states = {
    'US_states': future_US_states_timeseries.US_states.values,
    'time': future_US_states_timeseries.time.values
}


corrected_future_countries_timeseries = xr.DataArray(data=np.zeros((178, 1092)), dims=dims, coords=coords)
corrected_future_US_states_timeseries = xr.DataArray(data=np.zeros((52, 1092)), dims=dims_states, coords=coords_states)

years_historical = pd.date_range(start='1971-01', periods=hist_countries_timeseries.sizes['time'], freq='M')
years_future = pd.date_range(start='2010-01', periods=future_countries_timeseries.sizes['time'], freq='M')

## Apply the necessary correction algorithm:

In [14]:
apply_the_correction_algorithm_at_country_and_US_state_level_v3(countries_names, states_names, hist_countries_timeseries, future_countries_timeseries, hist_US_states_timeseries, future_US_states_timeseries, corrected_future_countries_timeseries, corrected_future_US_states_timeseries, years_future, sector="Irrigation", use_bias_adjustment=True, apply_amplitude_correction=True, number_of_years_to_use_for_amplitude_reference_N=5, apply_iterative_rescaling_to_correct_for_negative_values=True, minimum_future_ratio=0.1)

Fiji
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Calculation completed at 0%
Tanzania
No negative values found, no rescaling needed.
Final bias adjustment was completed.
W. Sahara
Negative values found, iterative rescaling applied.
Historical middle point is zero, skipping this country.
Final bias adjustment was completed.
Canada
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Alaska
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Alabama
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Arkansas
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Arizona
No negative values found, no rescaling needed.
Final bias adjustment was completed.
California
No negative values found, no rescaling needed.
Final bias adjustment was completed.
Colorado
No negative values found, no rescaling needed.
Final bias adjustment was

## Plot corrected national and US states timeseries:

In [15]:
plot_countries_monthly_timeseries_with_future_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/country_level/", hist_countries_timeseries, future_countries_timeseries, corrected_future_countries_timeseries, "Irrigation", countries_names)

All plots have been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/country_level/ directory.


In [16]:
plot_US_states_monthly_timeseries_with_future_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/US_state_level", hist_US_states_timeseries, future_US_states_timeseries, corrected_future_US_states_timeseries, "Irrigation", states_names)

All plots have been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/US_state_level directory.


## Plot orriginal and corrected global water withdrawal time-series:

In [17]:
plot_global_monthly_timeseries_with_future_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/global_level", hist_countries_timeseries, future_countries_timeseries, corrected_future_countries_timeseries, hist_US_states_timeseries, future_US_states_timeseries, corrected_future_US_states_timeseries, "Irrigation", countries_names, states_names)

Global plot has been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/global_level directory.


In [18]:
plot_global_monthly_timeseries_no_correction("/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/global_level", hist_countries_timeseries, future_countries_timeseries, hist_US_states_timeseries, future_US_states_timeseries, "Irrigation", countries_names, states_names)

Global plot has been created. You can check the results in the /vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/timeseries_after_correction/global_level directory.


## Visually inspect corrected national, US states and global timeseries:

Through visual inspection we can confirm a perfect harmonization between the past and future datasets.

# Check for any negative values

In [19]:
# Check for negative values in corrected_future_countries_timeseries
check_negative_values(corrected_future_countries_timeseries, countries_names, entity_type="country")

# Check for negative values in corrected_future_US_states_timeseries
check_negative_values(corrected_future_US_states_timeseries, states_names, entity_type="US_states")


No negative values found in Country dataset.
No negative values found in Us_states dataset.


# Save the time-series as .csv files

In [20]:
import xarray as xr
import pandas as pd

# Assuming the following DataArrays are already defined:
# hist_countries_timeseries: xarray.DataArray with dims (country: 178, time: 480)
# corrected_future_countries_timeseries: xarray.DataArray with dims (country: 178, time: 1092)
# countries_names: dictionary mapping indices to country names

# Concatenate the historical and future time-series along the time dimension
combined_timeseries = xr.concat([hist_countries_timeseries, corrected_future_countries_timeseries], dim='time')

# Loop over each country and save the corresponding time-series to a CSV file
for country_index in range(len(countries_names)):
    # Extract the data for the current country
    country_data = combined_timeseries.sel(country=country_index)
    
    # Convert to DataFrame
    country_df = country_data.to_dataframe(name='value').reset_index()
    
    # Map the country index to its name
    country_name = countries_names[country_index]
    country_df['country'] = country_name
    
    # Save to CSV with the country name as the filename
    filename = "./csv_files_of_countries_and_us_states_timeseries/irrigation_" + f"{country_name}_timeseries.csv"
    country_df.to_csv(filename, index=False)
    
    print(f"Saved {filename}")


Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Fiji_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Tanzania_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_W. Sahara_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Canada_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_United States of America_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Kazakhstan_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Uzbekistan_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Papua New Guinea_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Indonesia_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Argentina_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigatio

In [21]:
import xarray as xr
import pandas as pd

# Assuming the following DataArrays are already defined:
# hist_US_states_timeseries: xarray.DataArray with dims (state: N, time: 480)
# corrected_future_US_states_timeseries: xarray.DataArray with dims (state: N, time: 1092)
# states_names: dictionary mapping indices to state names

# Concatenate the historical and future time-series along the time dimension
combined_timeseries = xr.concat([hist_US_states_timeseries, corrected_future_US_states_timeseries], dim='time')

# Loop over each state and save the corresponding time-series to a CSV file
for state_index in range(len(states_names)):
    # Extract the data for the current state
    state_data = combined_timeseries.sel(US_states=state_index)
    
    # Convert to DataFrame
    state_df = state_data.to_dataframe(name='value').reset_index()
    
    # Map the state index to its name
    state_name = states_names[state_index]
    state_df['US_states'] = state_name
    
    # Save to CSV with the state name as the filename
    filename = "./csv_files_of_countries_and_us_states_timeseries/irrigation_" + f"{state_name}_timeseries.csv"
    state_df.to_csv(filename, index=False)
    
    print(f"Saved {filename}")


Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Alaska_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Alabama_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Arkansas_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Arizona_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_California_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Colorado_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Connecticut_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_District of Columbia_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Delaware_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Florida_timeseries.csv
Saved ./csv_files_of_countries_and_us_states_timeseries/irrigation_Georgia_tim

# Downscale the national and US states data

After correcting the time-series at natioanal level, we should update the individual gridcells values accordingly. To do so, we apply the same gridcells weights as in the original future water use dataset to perform the spatial downscaling of the new national time series.

In [22]:
path_to_save_corrected_gridded_data = "/user/brussel/105/vsc10535/SocioHydroLayers/input/sector_demand_future_link/ssp2_rcp26_gfdl/other_sectors/nc_format/yearly_format/irrigation"
downscale_future_corrected_national_and_US_states_timeseries_to_gridded_format_and_save_it(future_gridded_data, path_to_save_corrected_gridded_data, mask_countries, states_indices, mask_US_states, countries_indices, corrected_future_countries_timeseries, corrected_future_US_states_timeseries, "total_withdrawal_irrigation", "twdirrig")

Year completed: 2010
Year completed: 2020
Year completed: 2030
Year completed: 2040
Year completed: 2050
Year completed: 2060
Year completed: 2070
Year completed: 2080
Year completed: 2090
Year completed: 2100
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2010.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2011.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2012.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2013.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2014.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2015.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2016.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2017.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2018.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2019.nc
Saved twdirrig_m3_per_day_with_corrected_national_time_series_2020.nc
Saved twdirrig_m3_pe

# Plot the spatial consistency:

In [23]:
# Historical data processing
historical_directory = '/user/brussel/105/vsc10535/SocioHydroLayers/input/sector_demand_link/processed/yearly_format/irrigation/ensemble_mean/'
historical_pattern = "ensemble_mean_"
historical_data = process_files_annual_average(historical_directory, historical_pattern)

# Future data processing
future_directory = '/user/brussel/105/vsc10535/our_vo/SocioHydroLayers/input/sector_demand_future/ssp2_rcp26_gfdl/other_sectors/nc_format/yearly_format/irrigation/'
future_pattern = 'twdirr_m3_per_day'
future_data = process_files_annual_average(future_directory, future_pattern)

# Corrected future data processing
corrected_future_pattern = 'twdirrig_m3_per_day_with_corrected_national_time_series'
corrected_future_data = process_files_annual_average(future_directory, corrected_future_pattern)

A xarray.Dataset corresponding to the annualy aggregated data in the given path was succesfully generated.
A xarray.Dataset corresponding to the annualy aggregated data in the given path was succesfully generated.
A xarray.Dataset corresponding to the annualy aggregated data in the given path was succesfully generated.


In [24]:
def plot_water_withdrawal_changes(historical_data, original_future_data, corrected_future_data, variable_name_hist, variable_name_future, variable_name_for_the_title, path_save_plot):
    map_proj = ccrs.Robinson(central_longitude=0)
    
    # Create a figure with 1 row and 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), subplot_kw={'projection': map_proj})
    
    # First figure: Change for 2010/2011 with the original data
    ax1 = axes[0]
    ww_2010 = historical_data.sel(time='2010')[variable_name_hist][0]
    ww_2011_original = original_future_data.sel(time='2011')[variable_name_future][0]
    delta_original = (ww_2011_original - ww_2010) / ww_2010 * 100
    
    ax1.axis('off')
    ax1.coastlines(color='lightgray', linewidth=0.5)
    delta_original.plot(ax=ax1, cbar_kwargs={'fraction': 0.02, 'pad': 0.04, 'extend': 'both'}, cmap='RdBu', vmin=-100, vmax=100, transform=ccrs.PlateCarree(), add_colorbar=True, add_labels=False)
    ax1.set_title(f'$\Delta$ {variable_name_for_the_title} 2010-2011 (Original) (%)', loc='right', fontsize=20)
    
    # Second figure: Change for 2010/2011 with the corrected data
    ax2 = axes[1]
    ww_2011_corrected = corrected_future_data.sel(time='2011')[variable_name_future][0]
    delta_corrected = (ww_2011_corrected - ww_2010) / ww_2010 * 100
    
    ax2.axis('off')
    ax2.coastlines(color='lightgray', linewidth=0.5)
    delta_corrected.plot(ax=ax2, cbar_kwargs={'fraction': 0.02, 'pad': 0.04, 'extend': 'both'}, cmap='RdBu', vmin=-100, vmax=100, transform=ccrs.PlateCarree(), add_colorbar=True, add_labels=False)
    ax2.set_title(f'$\Delta$ {variable_name_for_the_title} 2010-2011 (Corrected) (%)', loc='right', fontsize=20)
    
    fig.tight_layout()
    plt.savefig(path_save_plot, format='pdf')


In [25]:
path_save_plot = "/vscmnt/brussel_pixiu_data/_data_brussel/105/vsc10535/SocioHydroLayers/source/sector_demand_data_harmonization/Final_data_harmonization_codebase_ssp2_rcp26_gfdl/plots/spatial_consistency_maps/relative_change_in_irrigation_water_withdrawal_original.pdf"
plot_water_withdrawal_changes(historical_data, future_data, corrected_future_data, "h08_withd_irrig", "total_withdrawal_irrigation", "Irrigation Withdrawal", path_save_plot) 

In [26]:
future_data

<xarray.Dataset>
Dimensions:                      (lon: 720, lat: 280, time: 91)
Coordinates:
  * lon                          (lon) float64 -179.8 -179.2 ... 179.2 179.8
  * lat                          (lat) float64 -55.75 -55.25 ... 83.25 83.75
  * time                         (time) datetime64[ns] 2010-12-31 ... 2100-12-31
Data variables:
    total_withdrawal_irrigation  (time, lat, lon) float64 nan nan ... nan nan